In [1]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates, Plots, TimeSeries, JSON

In [2]:
directory = joinpath(homedir(), "research", "GADS_Processed", "trajectories")
files = glob("*_state_Texas_raw.csv", directory)
x = Dict()
mydict = Dict([("A", 1), ("RS", 1), ("NC", 1),  ("IR", 1), ("MB", 1), ("F", 2), ("U1", 2), ("U2" , 2), ("U3" , 2), ("D1", 2), ("D2", 2), ("D3", 2), ("SF", 2), ("PO", 3), ("PE", 3), ("PD", 3), ("DP", 3), ("MO", 4), ("ME", 4), ("DM", 4), ("D4", 4), ("RU", 5), ("DR", 5), ("TC", 5)])
agedict = Dict([(1, DateTime(1980, 1, 1)), (2, DateTime(1985, 1, 1)), (3, DateTime(1995, 1, 1)), (4, DateTime(2005, 1, 1)), (5, DateTime(2015, 1, 1)), (6, DateTime(2020, 1, 1))])
types_dict = Dict("EventStartDT" => DateTime, "EventEndDT" => DateTime)
rating_dict = Dict([(1, 50), (2, 150), (3, 200)]);

In [ ]:
i = 1
for f in files

    mymatch = match(r"/home/emco4286/research/GADS_Processed/trajectories/gen_(\d+)_type_(\d+)_dtgrp_(\d+)_rating_(\d+)_state_Texas_raw.csv", f)
    id = parse(Int32, mymatch[1])
    type = parse(Int32, mymatch[2])
    dtgrp = parse(Int32, mymatch[3])
    rating = parse(Int32, mymatch[4])

    df = DataFrame(CSV.File(f))

    x[i] = Dict([("id", id), ("type", type), ("dtgrp", dtgrp), ("rating", rating), ("traj", df)])
    i +=1
end

In [4]:
types = unique(x[j]["type"] for j in keys(x))

idx_by_type = [[j for j in keys(x) if x[j]["type"] == t] for t in types]

timearrays = [[x[j]["traj"] for j in i] for i in idx_by_type]

one_ta = [reduce((i, j) -> outerjoin(i, j, on=:x, makeunique=true), timearrays[k]) for k in 1:1:13]

one_ta = [j[begin:96409, :] for j in one_ta];

In [5]:
using DataStructures

In [6]:
acc= [[counter(j) for j in eachrow(k[!, names(k)[2:end]])] for k in one_ta]
num_units = [[k[0.0] + k[1.0]  for k in j] for j in acc];

In [7]:
label = reshape(types, (1,13))
colnames = reshape(Symbol.(label), (13,))

1×13 Matrix{Int32}:
 861  851  300  852  100  650  850  500  200  800  860  862  400

In [28]:
my_num_units = reduce(hcat, num_units)
cf = DataFrame(my_num_units, colnames)
cf[!, :x] = one_ta[1][!, :x]
CSV.write("texas_number_of_active_units_by_type.csv", cf)

"texas_number_of_active_units_by_type.csv"

In [8]:
available_capacity = [sum.(skipmissing.(eachrow(k[!, names(k)[2:end]]))) for k in one_ta];

In [9]:
pct_of_units = [available_capacity[k] .* 100 ./ num_units[k] for k in 1:1:13];

In [10]:
y_matrix = 100 .- (reduce(hcat, pct_of_units))

96409×13 Matrix{Float64}:
  4.87805  7.14286  5.95238   7.89474  …  33.3333    0.0   9.09091  100.0
  4.87805  7.14286  5.95238   7.89474     33.3333    0.0   9.09091  100.0
  4.87805  7.14286  5.95238   7.89474     33.3333    0.0   9.09091  100.0
  4.87805  7.14286  5.95238   7.89474     33.3333    0.0   9.09091  100.0
  4.87805  7.14286  5.95238   7.89474     33.3333    0.0   9.09091  100.0
  4.87805  7.14286  5.95238   7.89474  …  33.3333    0.0   9.09091  100.0
  4.87805  7.14286  5.95238   7.89474     33.3333    0.0   9.09091  100.0
  4.87805  7.14286  5.95238   7.89474     33.3333    0.0   9.09091  100.0
  4.87805  7.14286  5.95238   7.89474     33.3333    0.0   9.09091  100.0
  4.87805  7.14286  5.95238   7.89474     33.3333    0.0   9.09091  100.0
  ⋮                                    ⋱             ⋮              
 22.7273   5.45455  4.26829  11.5385   …   0.0     100.0  33.3333    11.1111
 22.7273   5.45455  4.26829  11.5385       0.0     100.0  33.3333    11.1111
 22.7273   

In [ ]:
gf = DataFrame(y_matrix, colnames)
gf[!, :x] = one_ta[1][!, :x]
CSV.write("texas_available_capacity_pct_by_unit_type.csv",  gf)

Row,861,851,300,852,100,650,850,500,200,800,860,862,400
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,4.87805,7.14286,5.95238,7.89474,14.2857,0.0,3.0303,33.3333,0.0,33.3333,0.0,9.09091,100.0
2,4.87805,7.14286,5.95238,7.89474,14.2857,0.0,3.0303,33.3333,0.0,33.3333,0.0,9.09091,100.0
3,4.87805,7.14286,5.95238,7.89474,14.2857,0.0,3.0303,33.3333,0.0,33.3333,0.0,9.09091,100.0
4,4.87805,7.14286,5.95238,7.89474,14.2857,0.0,3.0303,33.3333,0.0,33.3333,0.0,9.09091,100.0
5,4.87805,7.14286,5.95238,7.89474,14.2857,0.0,3.0303,33.3333,0.0,33.3333,0.0,9.09091,100.0
6,4.87805,7.14286,5.95238,7.89474,15.0794,0.0,3.0303,33.3333,0.0,33.3333,0.0,9.09091,100.0
7,4.87805,7.14286,5.95238,7.89474,14.2857,0.0,3.0303,33.3333,0.0,33.3333,0.0,9.09091,100.0
8,4.87805,7.14286,5.95238,7.89474,14.2857,0.0,3.0303,33.3333,0.0,33.3333,0.0,9.09091,100.0
9,4.87805,7.14286,5.95238,7.89474,13.4921,0.0,3.0303,33.3333,0.0,33.3333,0.0,9.09091,100.0
